In [1]:
import lightgbm
import pickle
import pandas as pd
import numpy as np
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

In [3]:
# validation set
x_val = x.loc[x['srch_id']%10 == 0]
x = x.drop(x_val.index)

In [4]:
# drop columns which cannot be used for both train and val
x.drop(['click_bool', 'booking_bool', 'date_time', 'random_bool'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time', 'random_bool'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,prop_review_score_srch_id_std,prop_location_score1_srch_id_std,prop_location_score2_srch_id_mean,prop_location_score2_srch_id_std,prop_location_score2_srch_id_median,price_usd_srch_destination_id_median,prop_location_score2_srch_destination_id_mean,prop_location_score2_srch_destination_id_std,prop_location_score2_srch_destination_id_median,target
0,30,24,216,219,13155,3,3.5,1,4.76,0.3273,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
1,30,24,216,219,137592,3,4.0,1,4.61,0.2374,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
2,30,24,216,219,136754,3,0.0,0,4.81,0.0000,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
3,30,24,216,219,133729,3,3.5,1,4.77,0.3932,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
4,30,24,216,219,130968,4,4.5,1,4.81,0.3536,...,0.886965,0.809980,0.300107,0.153532,0.3588,198.67,0.325985,0.133087,0.366,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503647,332750,5,219,219,58095,2,3.0,0,1.39,0.0004,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503648,332750,5,219,219,58358,3,4.5,1,2.94,0.0074,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503649,332750,5,219,219,64324,2,4.0,1,0.00,0.0003,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0
503650,332750,5,219,219,40869,4,4.0,1,2.08,0.0125,...,0.636634,1.092423,0.039761,0.043743,0.0189,140.00,0.044633,0.039330,0.035,0


In [5]:
# make targets
y_val = x_val.target.squeeze()
y = x.target.squeeze()

# remove target column from train and val
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [6]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [7]:
# remove search id
x.drop(['srch_id'], axis=1, inplace=True)
x_val.drop(['srch_id'], axis=1, inplace=True)

In [8]:
pd.set_option('display.max_columns', None)
x.head(n=1)

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,agg_comp_rate,agg_comp_perc,agg_comp_inv,price_usd_prop_id_mean,price_usd_prop_id_std,price_usd_prop_id_median,prop_starrating_prop_id_mean,prop_review_score_prop_id_mean,prop_location_score1_prop_id_mean,prop_location_score1_prop_id_median,prop_location_score2_prop_id_mean,prop_location_score2_prop_id_std,prop_location_score2_prop_id_median,price_usd_srch_id_mean,price_usd_srch_id_std,price_usd_srch_id_median,prop_starrating_srch_id_std,prop_review_score_srch_id_std,prop_location_score1_srch_id_std,prop_location_score2_srch_id_mean,prop_location_score2_srch_id_std,prop_location_score2_srch_id_median,price_usd_srch_destination_id_median,prop_location_score2_srch_destination_id_mean,prop_location_score2_srch_destination_id_std,prop_location_score2_srch_destination_id_median
0,12,187,219,893,3,3.5,1,2.83,0.0438,4.95,104.77,0,23246,1,0,4,0,1,1,-1.0,0.0,0.0,0.0,118.758742,17.778734,118.0,3,3.5,2.83,2.83,0.053879,0.031432,0.0438,163.718929,96.355255,139.05,0.766356,1.109572,0.518734,0.045493,0.047216,0.02255,118.46,0.037571,0.043714,0.0155


In [9]:
categorical_columns = [0,1,2,3,6,11,12,18]#[1,2,3,4,7,12,13,19]
x.iloc[:,categorical_columns]

,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_brand_bool,promotion_flag,srch_destination_id,srch_saturday_night_bool
0,12,187,219,893,1,0,23246,1
1,12,187,219,10404,1,0,23246,1
2,12,187,219,21315,1,0,23246,1
3,12,187,219,27348,1,0,23246,1
4,12,187,219,29604,1,0,23246,1
...,...,...,...,...,...,...,...,...
4958342,5,219,219,77700,1,0,16974,0
4958343,5,219,219,88083,1,0,16974,0
4958344,5,219,219,94508,1,0,16974,0
4958345,5,219,219,128360,1,0,16974,0


In [ ]:
# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=3000, extra_trees=True,
                         seed=12345, deterministic=True)#, #learning_rate = 0.05)#, num_leaves=50) 

## nieuw: extra_trees
# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=500,
    categorical_feature=categorical_columns
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 6, 11, 12, 18]
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] num_iterations is set=3000, num_boost_round=3000 will be ignored. Current value: num_iterations=3000


/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's ndcg@1: 0.124818	valid_0's ndcg@2: 0.180994	valid_0's ndcg@3: 0.22083	valid_0's ndcg@4: 0.250918	valid_0's ndcg@5: 0.275416
Training until validation scores don't improve for 500 rounds
[2]	valid_0's ndcg@1: 0.147575	valid_0's ndcg@2: 0.211774	valid_0's ndcg@3: 0.253272	valid_0's ndcg@4: 0.285791	valid_0's ndcg@5: 0.308877
[3]	valid_0's ndcg@1: 0.159471	valid_0's ndcg@2: 0.227065	valid_0's ndcg@3: 0.268774	valid_0's ndcg@4: 0.300413	valid_0's ndcg@5: 0.324758
[4]	valid_0's ndcg@1: 0.162403	valid_0's ndcg@2: 0.231806	valid_0's ndcg@3: 0.273389	valid_0's ndcg@4: 0.304466	valid_0's ndcg@5: 0.330355
[5]	valid_0's ndcg@1: 0.170277	valid_0's ndcg@2: 0.240175	valid_0's ndcg@3: 0.281675	valid_0's ndcg@4: 0.313224	valid_0's ndcg@5: 0.338514
[6]	valid_0's ndcg@1: 0.172251	valid_0's ndcg@2: 0.243118	valid_0's ndcg@3: 0.285431	valid_0's ndcg@4: 0.317155	valid_0's ndcg@5: 0.342715
[7]	valid_0's ndcg@1: 0.17319	valid_0's ndcg@2: 0.244146	valid_0's ndcg@3: 0.287983	valid_0's ndcg@4: 0.3

[60]	valid_0's ndcg@1: 0.200132	valid_0's ndcg@2: 0.278281	valid_0's ndcg@3: 0.323944	valid_0's ndcg@4: 0.355747	valid_0's ndcg@5: 0.379871
[61]	valid_0's ndcg@1: 0.200477	valid_0's ndcg@2: 0.278406	valid_0's ndcg@3: 0.324552	valid_0's ndcg@4: 0.356161	valid_0's ndcg@5: 0.380329
[62]	valid_0's ndcg@1: 0.200875	valid_0's ndcg@2: 0.278589	valid_0's ndcg@3: 0.324573	valid_0's ndcg@4: 0.356367	valid_0's ndcg@5: 0.380687
[63]	valid_0's ndcg@1: 0.200878	valid_0's ndcg@2: 0.278601	valid_0's ndcg@3: 0.324532	valid_0's ndcg@4: 0.356452	valid_0's ndcg@5: 0.380387
[64]	valid_0's ndcg@1: 0.20157	valid_0's ndcg@2: 0.279083	valid_0's ndcg@3: 0.325352	valid_0's ndcg@4: 0.357186	valid_0's ndcg@5: 0.380949
[65]	valid_0's ndcg@1: 0.201418	valid_0's ndcg@2: 0.279159	valid_0's ndcg@3: 0.32565	valid_0's ndcg@4: 0.357302	valid_0's ndcg@5: 0.381221
[66]	valid_0's ndcg@1: 0.202013	valid_0's ndcg@2: 0.27942	valid_0's ndcg@3: 0.326194	valid_0's ndcg@4: 0.357892	valid_0's ndcg@5: 0.38165
[67]	valid_0's ndcg@1: 0

...

[1004]	valid_0's ndcg@1: 0.213103	valid_0's ndcg@2: 0.293251	valid_0's ndcg@3: 0.341609	valid_0's ndcg@4: 0.373353	valid_0's ndcg@5: 0.397917
[1005]	valid_0's ndcg@1: 0.213103	valid_0's ndcg@2: 0.293345	valid_0's ndcg@3: 0.341563	valid_0's ndcg@4: 0.373351	valid_0's ndcg@5: 0.397926
[1006]	valid_0's ndcg@1: 0.213202	valid_0's ndcg@2: 0.293406	valid_0's ndcg@3: 0.341574	valid_0's ndcg@4: 0.373404	valid_0's ndcg@5: 0.397941
[1007]	valid_0's ndcg@1: 0.213202	valid_0's ndcg@2: 0.293345	valid_0's ndcg@3: 0.34134	valid_0's ndcg@4: 0.373361	valid_0's ndcg@5: 0.397878
[1008]	valid_0's ndcg@1: 0.213252	valid_0's ndcg@2: 0.293363	valid_0's ndcg@3: 0.341383	valid_0's ndcg@4: 0.373426	valid_0's ndcg@5: 0.397885
[1009]	valid_0's ndcg@1: 0.213202	valid_0's ndcg@2: 0.293438	valid_0's ndcg@3: 0.341384	valid_0's ndcg@4: 0.373384	valid_0's ndcg@5: 0.397882
[1010]	valid_0's ndcg@1: 0.213204	valid_0's ndcg@2: 0.293345	valid_0's ndcg@3: 0.341317	valid_0's ndcg@4: 0.37338	valid_0's ndcg@5: 0.397881
[1011]	v

[1062]	valid_0's ndcg@1: 0.212605	valid_0's ndcg@2: 0.292594	valid_0's ndcg@3: 0.3408	valid_0's ndcg@4: 0.372678	valid_0's ndcg@5: 0.397298
[1063]	valid_0's ndcg@1: 0.212506	valid_0's ndcg@2: 0.29267	valid_0's ndcg@3: 0.340907	valid_0's ndcg@4: 0.372741	valid_0's ndcg@5: 0.397363
[1064]	valid_0's ndcg@1: 0.212408	valid_0's ndcg@2: 0.292696	valid_0's ndcg@3: 0.340885	valid_0's ndcg@4: 0.372698	valid_0's ndcg@5: 0.397301
[1065]	valid_0's ndcg@1: 0.212357	valid_0's ndcg@2: 0.292815	valid_0's ndcg@3: 0.340952	valid_0's ndcg@4: 0.372731	valid_0's ndcg@5: 0.397392
[1066]	valid_0's ndcg@1: 0.212307	valid_0's ndcg@2: 0.292795	valid_0's ndcg@3: 0.340785	valid_0's ndcg@4: 0.372717	valid_0's ndcg@5: 0.397318
[1067]	valid_0's ndcg@1: 0.212406	valid_0's ndcg@2: 0.292829	valid_0's ndcg@3: 0.340895	valid_0's ndcg@4: 0.372785	valid_0's ndcg@5: 0.397385
[1068]	valid_0's ndcg@1: 0.212405	valid_0's ndcg@2: 0.292891	valid_0's ndcg@3: 0.340859	valid_0's ndcg@4: 0.372748	valid_0's ndcg@5: 0.397386
[1069]	va

[1121]	valid_0's ndcg@1: 0.212998	valid_0's ndcg@2: 0.29277	valid_0's ndcg@3: 0.341052	valid_0's ndcg@4: 0.372513	valid_0's ndcg@5: 0.397135
[1122]	valid_0's ndcg@1: 0.213097	valid_0's ndcg@2: 0.292931	valid_0's ndcg@3: 0.341163	valid_0's ndcg@4: 0.372539	valid_0's ndcg@5: 0.397181
[1123]	valid_0's ndcg@1: 0.213097	valid_0's ndcg@2: 0.292806	valid_0's ndcg@3: 0.341113	valid_0's ndcg@4: 0.37248	valid_0's ndcg@5: 0.397149
[1124]	valid_0's ndcg@1: 0.213095	valid_0's ndcg@2: 0.292954	valid_0's ndcg@3: 0.341137	valid_0's ndcg@4: 0.372561	valid_0's ndcg@5: 0.397179
[1125]	valid_0's ndcg@1: 0.213095	valid_0's ndcg@2: 0.292954	valid_0's ndcg@3: 0.341137	valid_0's ndcg@4: 0.372582	valid_0's ndcg@5: 0.397182
[1126]	valid_0's ndcg@1: 0.213146	valid_0's ndcg@2: 0.292973	valid_0's ndcg@3: 0.341168	valid_0's ndcg@4: 0.372573	valid_0's ndcg@5: 0.397199
[1127]	valid_0's ndcg@1: 0.213146	valid_0's ndcg@2: 0.292848	valid_0's ndcg@3: 0.341142	valid_0's ndcg@4: 0.372624	valid_0's ndcg@5: 0.397193
[1128]	v

[1179]	valid_0's ndcg@1: 0.212702	valid_0's ndcg@2: 0.292236	valid_0's ndcg@3: 0.341065	valid_0's ndcg@4: 0.372413	valid_0's ndcg@5: 0.396968
[1180]	valid_0's ndcg@1: 0.212653	valid_0's ndcg@2: 0.292093	valid_0's ndcg@3: 0.34086	valid_0's ndcg@4: 0.372367	valid_0's ndcg@5: 0.396923
[1181]	valid_0's ndcg@1: 0.212653	valid_0's ndcg@2: 0.292062	valid_0's ndcg@3: 0.340929	valid_0's ndcg@4: 0.372351	valid_0's ndcg@5: 0.396926
[1182]	valid_0's ndcg@1: 0.212504	valid_0's ndcg@2: 0.292182	valid_0's ndcg@3: 0.340959	valid_0's ndcg@4: 0.372295	valid_0's ndcg@5: 0.39687
[1183]	valid_0's ndcg@1: 0.212554	valid_0's ndcg@2: 0.292201	valid_0's ndcg@3: 0.340914	valid_0's ndcg@4: 0.372433	valid_0's ndcg@5: 0.396922
[1184]	valid_0's ndcg@1: 0.212554	valid_0's ndcg@2: 0.292232	valid_0's ndcg@3: 0.340946	valid_0's ndcg@4: 0.372443	valid_0's ndcg@5: 0.396951
[1185]	valid_0's ndcg@1: 0.212702	valid_0's ndcg@2: 0.292236	valid_0's ndcg@3: 0.34094	valid_0's ndcg@4: 0.372565	valid_0's ndcg@5: 0.396921
[1186]	va

[1238]	valid_0's ndcg@1: 0.212063	valid_0's ndcg@2: 0.292252	valid_0's ndcg@3: 0.340983	valid_0's ndcg@4: 0.372483	valid_0's ndcg@5: 0.396721
[1239]	valid_0's ndcg@1: 0.212112	valid_0's ndcg@2: 0.292271	valid_0's ndcg@3: 0.341029	valid_0's ndcg@4: 0.37259	valid_0's ndcg@5: 0.396695
[1240]	valid_0's ndcg@1: 0.212013	valid_0's ndcg@2: 0.292251	valid_0's ndcg@3: 0.340932	valid_0's ndcg@4: 0.37254	valid_0's ndcg@5: 0.396613
[1241]	valid_0's ndcg@1: 0.212061	valid_0's ndcg@2: 0.292269	valid_0's ndcg@3: 0.340889	valid_0's ndcg@4: 0.372699	valid_0's ndcg@5: 0.396639
[1242]	valid_0's ndcg@1: 0.212061	valid_0's ndcg@2: 0.292237	valid_0's ndcg@3: 0.340833	valid_0's ndcg@4: 0.372664	valid_0's ndcg@5: 0.396566
[1243]	valid_0's ndcg@1: 0.212012	valid_0's ndcg@2: 0.29225	valid_0's ndcg@3: 0.340846	valid_0's ndcg@4: 0.372677	valid_0's ndcg@5: 0.396559
[1244]	valid_0's ndcg@1: 0.211962	valid_0's ndcg@2: 0.29217	valid_0's ndcg@3: 0.340876	valid_0's ndcg@4: 0.372633	valid_0's ndcg@5: 0.39647
[1245]	vali

In [ ]:
gbm.get_params()

In [ ]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [ ]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [ ]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop(['date_time', 'random_bool'], axis=1, inplace=True)
srch_ids = test['srch_id']
test.drop('srch_id', axis=1, inplace=True)

In [ ]:
test

In [ ]:
predictions = model.predict(test)

In [ ]:
len(predictions)

In [ ]:
#submission = test[['srch_id', 'prop_id']]
submission = pd.concat([srch_ids, test['prop_id']], axis=1)
submission['pred'] = predictions

In [ ]:
submission.head(n=100)

In [ ]:
del test
gc.collect()

In [ ]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [ ]:
submission.head(n=20)

In [ ]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)